In [1]:
import os
import sys

os.chdir("/teamspace/studios/this_studio/nurse-helper-chatbot")

sys.path.append(os.getcwd())

In [2]:
from llm import llm
from graph import graph
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate


CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher statement to query a graph database.
Instructions:
- Use only the provided relationship types and properties in the schema.
- Do not use any other relationship types or properties that are not provided.
- Ensure that the generated Cypher statements are syntactically correct and relevant to the question.
- Do not include any explanations or apologies in your responses.
- Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
- Do not include any text except the generated Cypher statement.

Do not return entire nodes or embedding properties.

Fine Tuning:

Example Cypher Statements:

1. To find physicians by partial or exact name match with hospital details
```
MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE p.Physician_Name CONTAINS "Lim" OR p.Physician_Name = "Dr. Physician Name"
RETURN p.Physician_Name AS Name, 
       p.Specialization AS Specialization, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact AS PhysicianContactNumber,
       h.Hospital_Name AS HospitalName;

```

2. To find a physician by their license number and the hospital they work at
```
MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE p.License_Number = 48720
RETURN p.Physician_Name AS Name, 
       p.Specialization AS Specialization, 
       p.Physician_Contact AS PhysicianContactNumber,
       h.`Hospital Name` AS HospitalName;
```
3. 
```
MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE h.Hospital_Name IN $hospitalNames AND 
      p.Specialization IN $specializations
RETURN p.Physician_Name AS Name, 
       p.Specialization AS Specialization, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact AS PhysicianContactNumber,
       h.Hospital_Name AS HospitalName;
```
Schema:
{schema}

Question:
{question}

Cypher Query:
"""


physician_cypher_prompt = PromptTemplate.from_template(CYPHER_GENERATION_TEMPLATE)

physician_cypher_qa = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    return_intermediate_steps=True,
    verbose=True,
    allow_dangerous_requests=True,
    validate_cypher=True,
    function_response_system="Response based on the Cypher Query and context",
    cypher_prompt=physician_cypher_prompt
)

/teamspace/studios/this_studio/nurse-helper-chatbot/graph.py:7: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [6]:
# Example query to invoke the chain
## Find doctors with 'Tan' in their name and where hospital they work for.
## Get information about Dr. Saraswathy Pillai and the hospital where they work.
## Show doctors whose name includes 'Lim' or exactly matches Dr. Ng Yee Siang, along with their hospital details.
## Find doctors with license number 48720 and the hospital they work at.
## List doctors specializing in Cardiology or Orthopedics at Penang General Hospital and Penang Adventist Hospital.

query = "Can u give me which sepcialization Dr. Siti Rahmah binti Abdul Latif?"
# Invoke the chain with the complete input data
response = physician_cypher_qa.run(query)

response



> Entering new GraphCypherQAChain chain...


Generated Cypher:

MATCH (p:physician)
WHERE p.Physician_Name = "Dr. Siti Rahmah binti Abdul Latif"
RETURN p.Specialization AS Specialization;

Full Context:
[{'Specialization': 'Orthopedics'}]

> Finished chain.


'Dr. Siti Rahmah binti Abdul Latif specializes in Orthopedics.'

In [5]:
from llm import llm
from graph import graph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.tools import Tool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain import hub
from utils import get_session_id
from tools.review import get_review
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler
import streamlit as st 


tools = [

    Tool.from_function(
        name="Physician Information",  
        description="Provide information about physician questions using Cypher and context from GraphCypherQAChain",
        func=physician_cypher_qa, 
    ),
    
]


def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

agent_prompt = PromptTemplate.from_template("""
You are a nursing expert providing information about nursing care.
Be as helpful as possible and return as much information as possible.
Only answer questions related to checkups, hospitals, or patients.

You must rely solely on the information provided in the context and the tools available to you. 
Do not use any pre-trained knowledge.



TOOLS:
------

You have access to the following tools:

{tools}

To use a tool, please follow this format:
```
Thought: Do I need to use a tool? Yes 
Action: [the action to take, should be one of {tool_names}] 
Action Input: [the input to the action] 
Observation: [the result of the action]
```

You **must** use the tools provided for every relevant question. 
Do not decline to use a tool, as it is essential for generating accurate responses.

When generating responses:
Always follow the required format strictly.
Use the actual Cypher examples from the `GraphCypherQAChain` to construct accurate queries.
Validate the information retrieved from the tools before providing a final answer.
If the information is insufficient, indicate that further clarification is needed.

Use the following format to provide a response:
```
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}
""")

agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    )

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)



def generate_response(user_input):
    """
    Create a handler that calls the Conversational agent
    and returns a response to be rendered in the UI
    """

    # Initialize the Streamlit callback handler
    st_callback = StreamlitCallbackHandler(st.container())

    # Call the agent and capture the response
    response = chat_agent.invoke(
        {"input": user_input},
        {"configurable": {"session_id": get_session_id()}, "callbacks": [st_callback]}
    )

    # Return the final output
    return response['output']

In [6]:
query = "List doctors specializing in Cardiology or Orthopedics at Penang General Hospital and Penang Adventist Hospital."

# Invoke the chain with the complete input data
response = generate_response(query)

response

2024-12-30 17:58:02.549 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:02.553 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:02.554 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:02.555 Session state does not function when running a script without `streamlit run`
2024-12-30 17:58:02.555 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:02.556 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:02.556 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:03.894 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58



> Entering new AgentExecutor chain...


2024-12-30 17:58:04.220 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:04.240 
  command:

    streamlit run /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-12-30 17:58:04.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:04.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:04.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:04.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:04.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:04.245 Thread 'MainThread': missing ScriptRunContext! This warning can 

```
Thought: Do I need to use a tool? Yes 
Action: Physician Information 
Action Input: {"hospitals": ["Penang General Hospital", "Penang Adventist Hospital"], "specialties": ["Cardiology", "Orthopedics"]} 

> Entering new GraphCypherQAChain chain...


Error in StreamlitCallbackHandler.on_llm_end callback: StreamlitAPIException()


...

Error in StreamlitCallbackHandler.on_tool_end callback: StreamlitAPIException()


{'query': '{"hospitals": ["Penang General Hospital", "Penang Adventist Hospital"], "specialties": ["Cardiology", "Orthopedics"]}', 'result': 'At Penang General Hospital, you can find the following specialists: Dr. Letchumi a/p Balakrishnan (Cardiology), Dr. Balakrishnan Menon (Orthopedics), Dr. Sharifah Nora binti Syed Ahmad (Cardiology), Dr. Ng Qian Yi (Orthopedics), Dr. Chee Seok Wing (Orthopedics), Dr. Loo Yen Fang (Cardiology), Dr. Cheah Wai Lum (Cardiology), and Dr. Low Hui Ting (Orthopedics). \n\nAt Penang Adventist Hospital, the orthopedic specialists available are Dr. Fikri Ahmad and Dr. Ooi Zhen Yi.', 'intermediate_steps': [{'query': '\nMATCH (p:physician)<-[:recruit]-(h:hospital)\nWHERE h.Hospital_Name IN ["Penang General Hospital", "Penang Adventist Hospital"] AND \n      p.Specialization IN ["Cardiology", "Orthopedics"]\nRETURN p.Physician_Name AS Name, \n       p.Specialization AS Specialization, \n       p.License_Number AS LicenseNumber, \n       p.Physician_Contact AS P

2024-12-30 17:58:10.876 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:10.877 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:10.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:10.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:10.886 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:10.887 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:10.888 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-30 17:58:10.888 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

```
Final Answer: At Penang General Hospital, the following specialists are available:
- Dr. Letchumi a/p Balakrishnan (Cardiology) - License Number: 48125, Contact: 03-5129887
- Dr. Balakrishnan Menon (Orthopedics) - License Number: 91109, Contact: 03-6459865
- Dr. Sharifah Nora binti Syed Ahmad (Cardiology) - License Number: 82414, Contact: 03-7786393
- Dr. Ng Qian Yi (Orthopedics) - License Number: 22293, Contact: 03-4145827
- Dr. Chee Seok Wing (Orthopedics) - License Number: 44822, Contact: 03-9925967
- Dr. Loo Yen Fang (Cardiology) - License Number: 10122, Contact: 03-1836810
- Dr. Cheah Wai Lum (Cardiology) - License Number: 98432, Contact: 03-5647382
- Dr. Low Hui Ting (Orthopedics) - License Number: 66534, Contact: 03-4895732

At Penang Adventist Hospital, the orthopedic specialists available are:
- Dr. Fikri Ahmad - License Number: 16470, Contact: 03-9440991
- Dr. Ooi Zhen Yi - License Number: 40160, Contact: 03-7919795
```

> Finished chain.


'At Penang General Hospital, the following specialists are available:\n- Dr. Letchumi a/p Balakrishnan (Cardiology) - License Number: 48125, Contact: 03-5129887\n- Dr. Balakrishnan Menon (Orthopedics) - License Number: 91109, Contact: 03-6459865\n- Dr. Sharifah Nora binti Syed Ahmad (Cardiology) - License Number: 82414, Contact: 03-7786393\n- Dr. Ng Qian Yi (Orthopedics) - License Number: 22293, Contact: 03-4145827\n- Dr. Chee Seok Wing (Orthopedics) - License Number: 44822, Contact: 03-9925967\n- Dr. Loo Yen Fang (Cardiology) - License Number: 10122, Contact: 03-1836810\n- Dr. Cheah Wai Lum (Cardiology) - License Number: 98432, Contact: 03-5647382\n- Dr. Low Hui Ting (Orthopedics) - License Number: 66534, Contact: 03-4895732\n\nAt Penang Adventist Hospital, the orthopedic specialists available are:\n- Dr. Fikri Ahmad - License Number: 16470, Contact: 03-9440991\n- Dr. Ooi Zhen Yi - License Number: 40160, Contact: 03-7919795\n```'